In [12]:
#Reading the train data and filtering it using the selected columns

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

usecols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
       
df_train = pd.read_csv('/Users/chidam/Desktop/santander_train_ver2.csv', usecols=usecols)
# sample = pd.read_csv('../input/sample_submission.csv')

df_train = df_train.drop_duplicates(['ncodpers'], keep='last')

df_train.fillna(0, inplace=True)

# df_train_altered=df_train[0:5000]



In [15]:
For every feature column I am considering all other columns excluding the 'ncodpers':Customer code and the 
current column

models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = df_train['ncodpers'].values
for c in df_train.columns:
    if c != 'ncodpers':
        print(c)
        y_train = df_train[c]
        x_train = df_train.drop([c, 'ncodpers'], 1)
        
        clf = LogisticRegression(max_iter=1000)
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        model_preds[c] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train))
        


ind_ahor_fin_ult1
0.7874078773631549
ind_aval_fin_ult1
0.9805999112508611
ind_cco_fin_ult1
0.6501703059911546
ind_cder_fin_ult1
0.7878490961699784
ind_cno_fin_ult1
0.9714745006880255
ind_ctju_fin_ult1
0.8655533428519265
ind_ctma_fin_ult1
0.6615849905745675
ind_ctop_fin_ult1
0.6127923195611304
ind_ctpp_fin_ult1
0.7223756470326701
ind_deco_fin_ult1
0.8880309784961913
ind_deme_fin_ult1
0.8675208347284112
ind_dela_fin_ult1
0.7818695493839363
ind_ecue_fin_ult1
0.7753438260664889
ind_fond_fin_ult1
0.8565788810165278
ind_hip_fin_ult1
0.9181826985630522
ind_plan_fin_ult1
0.8480840081714731
ind_pres_fin_ult1
0.7096128000685431
ind_reca_fin_ult1
0.8344127598096686
ind_tjcr_fin_ult1
0.9051585084424292
ind_valo_fin_ult1
0.8408031548656334
ind_viv_fin_ult1
0.7681046722725032
ind_nomina_ult1
0.9981799288741389
ind_nom_pens_ult1
0.9983109545977709
ind_recibo_ult1
0.8675065452325598


In [16]:
total number of records in the train data
len(df_train)

956645

In [17]:
# df_train_altered

In [18]:
Selecting the columns values for which the values were already 1, meaning the customers were already using the product.
In order to select the products that customers are not using currently we need to know the ones that they have been 
already using

already_active = {}
for row in df_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(df_train.columns[1:], row) if c[1] > 0]
    already_active[id] = active


    


In [19]:
already_active

{1049700.0: [],
 1055504.0: [],
 1045269.0: [],
 1042585.0: [],
 1047273.0: [],
 1048273.0: [],
 1047948.0: [],
 1045956.0: [],
 1065729.0: [],
 1064759.0: [],
 1064436.0: [],
 1069441.0: ['ind_recibo_ult1'],
 1069167.0: [],
 1059938.0: [],
 1058093.0: [],
 1058385.0: [],
 1062503.0: [],
 1060835.0: [],
 1025667.0: [],
 1023450.0: [],
 1023510.0: [],
 1024197.0: [],
 1029198.0: [],
 1026969.0: [],
 1026348.0: [],
 1026392.0: [],
 1027608.0: [],
 1027606.0: [],
 1016670.0: [],
 1014699.0: [],
 1022625.0: [],
 1023133.0: [],
 1022356.0: [],
 1018976.0: [],
 1018457.0: [],
 1038305.0: [],
 1038357.0: [],
 1040667.0: [],
 1042147.0: [],
 1039608.0: [],
 1031485.0: [],
 1030835.0: [],
 1031584.0: [],
 1034257.0: [],
 1035411.0: [],
 1104995.0: [],
 1104775.0: ['ind_nomina_ult1', 'ind_nom_pens_ult1'],
 1104365.0: [],
 1109496.0: [],
 1108729.0: [],
 1110361.0: [],
 1108235.0: [],
 1108420.0: [],
 1107857.0: [],
 1099721.0: [],
 1099219.0: [],
 1097196.0: [],
 1097562.0: [],
 1100120.0: [],
 

In [20]:
Selecting the top 7 product categories based on their probability scores and recommending the same to the customers

train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(df_train.columns[1:], p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:7]]
    train_preds[id] = preds
    


In [21]:
train_preds

{1049700: ['ind_cco_fin_ult1',
  'ind_ctop_fin_ult1',
  'ind_ecue_fin_ult1',
  'ind_cno_fin_ult1',
  'ind_ctju_fin_ult1',
  'ind_reca_fin_ult1',
  'ind_ctpp_fin_ult1'],
 1055504: ['ind_cco_fin_ult1',
  'ind_ctop_fin_ult1',
  'ind_ecue_fin_ult1',
  'ind_cno_fin_ult1',
  'ind_ctju_fin_ult1',
  'ind_reca_fin_ult1',
  'ind_ctpp_fin_ult1'],
 1045269: ['ind_cco_fin_ult1',
  'ind_ctop_fin_ult1',
  'ind_ecue_fin_ult1',
  'ind_cno_fin_ult1',
  'ind_ctju_fin_ult1',
  'ind_reca_fin_ult1',
  'ind_ctpp_fin_ult1'],
 1042585: ['ind_cco_fin_ult1',
  'ind_ctop_fin_ult1',
  'ind_ecue_fin_ult1',
  'ind_cno_fin_ult1',
  'ind_ctju_fin_ult1',
  'ind_reca_fin_ult1',
  'ind_ctpp_fin_ult1'],
 1047273: ['ind_cco_fin_ult1',
  'ind_ctop_fin_ult1',
  'ind_ecue_fin_ult1',
  'ind_cno_fin_ult1',
  'ind_ctju_fin_ult1',
  'ind_reca_fin_ult1',
  'ind_ctpp_fin_ult1'],
 1048273: ['ind_cco_fin_ult1',
  'ind_ctop_fin_ult1',
  'ind_ecue_fin_ult1',
  'ind_cno_fin_ult1',
  'ind_ctju_fin_ult1',
  'ind_reca_fin_ult1',
  'ind_ctp